# Lab 1

## 1. Task 1:

### (a) Explanation of your implementation and experimental setup for task 1.

First we observe the data and find that ID and name are less helpful for classification, latitude, longitude and mean_checkin_time are numerical values that may have an impact on the results, and review is text, which requires some processing before discussion.
After removing unnecessary features, some representations of category are replaced by numerical values.
The text is vectorized for word frequency using the CountVectorizer class, to find the frequency of each word.
Since the vocabulary size of the training set is different from that of the test set, the number of features is therefore different. Since the words that are not in the test set do not help the calculation, we choose to take the intersection of the two sets before performing the calculation.
The obtained matrix is merged into the other three features, and the data is normalized for Bayesian processing.

### (b) Summary and discussion of experimental results for task 1.

After calculation, we obtained a accuracy of model about 0.74, and it can be determined that this model can discriminate category, but there is still room for improvement.

### (c) Attached source codes for task 1.

In [1]:
from pathlib import Path
from time import time
import os
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [2]:
# reading the data
PATH_ROOT = os.getcwd()
PATH_TRAIN = os.path.join(PATH_ROOT, 'train.csv')
train_data = pd.read_csv(PATH_TRAIN,  header=0)
PATH_TEST = os.path.join(PATH_ROOT, 'test.csv')
test_data = pd.read_csv(PATH_TEST, header=0)
train_data.head()

,ID,name,latitude,longitude,mean_checkin_time,review,category
0,3007,The New Orleans Vampire Cafe,29.959033,-90.064036,17.0,Amazing service. Cool vibe. It's not spooky or...,Restaurants
1,1829,Ted's Frostop,29.947026,-90.113604,17.0,Breakfast here is great and there's never a hu...,Restaurants
2,298,The Will & The Way,29.957573,-90.065827,9.5,So glad that we stumbled in here! The cheesebu...,Restaurants
3,1245,Public Belt,29.946393,-90.063729,3.0,AMAZING! Try this place out. Great for some g...,Nightlife
4,2902,Phillys Cafe,29.941818,-90.094797,18.0,"WooHoo, best philly cheese staks I have had in...",Restaurants


In [3]:
# Filter for valuable features
train_data = train_data[['latitude','longitude','mean_checkin_time','review','category']]
test_data = test_data[['latitude','longitude','mean_checkin_time','review','category']]
train_data.head()

,latitude,longitude,mean_checkin_time,review,category
0,29.959033,-90.064036,17.0,Amazing service. Cool vibe. It's not spooky or...,Restaurants
1,29.947026,-90.113604,17.0,Breakfast here is great and there's never a hu...,Restaurants
2,29.957573,-90.065827,9.5,So glad that we stumbled in here! The cheesebu...,Restaurants
3,29.946393,-90.063729,3.0,AMAZING! Try this place out. Great for some g...,Nightlife
4,29.941818,-90.094797,18.0,"WooHoo, best philly cheese staks I have had in...",Restaurants


In [4]:
# Converting category labels to numbers
classes = train_data['category'].unique()
classes_types = CategoricalDtype(categories=classes)
train_data['category'] = train_data['category'].astype(classes_types).cat.codes.astype('long')
test_data['category'] = test_data['category'].astype(classes_types).cat.codes.astype('long')
train_data_review = train_data['review']
test_data_review = test_data['review']
train_data.head()

,latitude,longitude,mean_checkin_time,review,category
0,29.959033,-90.064036,17.0,Amazing service. Cool vibe. It's not spooky or...,0
1,29.947026,-90.113604,17.0,Breakfast here is great and there's never a hu...,0
2,29.957573,-90.065827,9.5,So glad that we stumbled in here! The cheesebu...,0
3,29.946393,-90.063729,3.0,AMAZING! Try this place out. Great for some g...,1
4,29.941818,-90.094797,18.0,"WooHoo, best philly cheese staks I have had in...",0


In [5]:
def type1(train_data,train_data_review,test_data,test_data_review):
    def process_review_CountVectorizer(data_review):
        '''
        Find the word frequency
        '''
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(data_review)
        T = vectorizer.get_feature_names()
        X = pd.DataFrame(X.toarray(),columns=T)
        X.index.name ='merge_index_unique'
        return X,T
    
    train_y = train_data['category'].to_numpy()
    test_y = test_data['category'].to_numpy()
    
    # Incorporate word frequencies into other features
    new1,T1 = process_review_CountVectorizer(train_data_review)
    new2,T2 = process_review_CountVectorizer(test_data_review)
    # 求训练集和测试集词汇量的交集，因为不在测试集的词汇训练了也没有用
    T = set(T1)&set(T2)
    T = list(T)
    # 合并训练集和测试集的特征
    new1 = new1[T]
    train_data.index.name ='merge_index_unique'
    train_data_new = pd.merge(train_data.reset_index(),new1.reset_index(),on='merge_index_unique')
    
    new2 = new2[T]
    test_data.index.name ='merge_index_unique'
    test_data_new = pd.merge(test_data.reset_index(),new2.reset_index(),on='merge_index_unique')

    train_x = train_data_new.drop(['category','review_x'], axis=1)
    test_x = test_data_new.drop(['category','review_x'], axis=1)
    
    # Normalize
    train_x = (train_x-train_x.min())/(train_x.max()-train_x.min())
    test_x = (test_x-test_x.min())/(test_x.max()-test_x.min())
    
    train_x = train_x.to_numpy()
    test_x = test_x.to_numpy()
    
    # Train and test the model
    nb_new = GaussianNB()
    nb_new.fit(train_x, train_y)
    print("The score is :")
    print(nb_new.score(test_x,test_y))
    print()
type1(train_data,train_data_review,test_data,test_data_review)

The score is :
0.7359307359307359



## 2. Task 2:

### (a) Explanation of your implementation and experimental setup for task 2.

Because of the sparse word frequency distribution, polynomial Bayesian is chosen for processing. 

### (b) Summary and discussion of experimental results for task 2.

After calculation, we obtained a accuracy of about 0.858, which has some accuracy improvement, but there is still room for improvement.

### (c) Attached source codes for task 2.

In [6]:
# reading the data
PATH_ROOT = os.getcwd()
PATH_TRAIN = os.path.join(PATH_ROOT, 'train.csv')
train_data = pd.read_csv(PATH_TRAIN,  header=0)
PATH_TEST = os.path.join(PATH_ROOT, 'test.csv')
test_data = pd.read_csv(PATH_TEST, header=0)
train_data = train_data[['latitude','longitude','mean_checkin_time','review','category']]
test_data = test_data[['latitude','longitude','mean_checkin_time','review','category']]
train_data.head()
classes = train_data['category'].unique()
classes_types = CategoricalDtype(categories=classes)
train_data['category'] = train_data['category'].astype(classes_types).cat.codes.astype('long')
test_data['category'] = test_data['category'].astype(classes_types).cat.codes.astype('long')
train_data_review = train_data['review']
test_data_review = test_data['review']

def type1(train_data,train_data_review,test_data,test_data_review):
    def process_review_CountVectorizer(data_review):
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(data_review)
        T = vectorizer.get_feature_names()
        X = pd.DataFrame(X.toarray(),columns=T)
        X.index.name ='merge_index_unique'
        return X,T
    
    train_y = train_data['category'].to_numpy()
    test_y = test_data['category'].to_numpy()
    
    # Incorporate word frequencies into other features
    new1,T1 = process_review_CountVectorizer(train_data_review)
    new2,T2 = process_review_CountVectorizer(test_data_review)
    # 求训练集和测试集词汇量的交集，因为不在测试集的词汇训练了也没有用
    T = set(T1)&set(T2)
    T = list(T)
    # 合并训练集和测试集的特征
    new1 = new1[T]
    train_data.index.name ='merge_index_unique'
    train_data = train_data.drop(['category','review'], axis=1)
    train_data_new = pd.merge(train_data.reset_index(),new1.reset_index(),on='merge_index_unique')
    
    new2 = new2[T]
    test_data.index.name ='merge_index_unique'
    test_data = test_data.drop(['category','review'], axis=1)
    test_data_new = pd.merge(test_data.reset_index(),new2.reset_index(),on='merge_index_unique')

    train_x = train_data_new
    test_x = test_data_new
    
    # Normalize
    train_x = (train_x-train_x.min())/(train_x.max()-train_x.min())
    test_x = (test_x-test_x.min())/(test_x.max()-test_x.min())
    
    train_x = train_x.to_numpy()
    test_x = test_x.to_numpy()
    
    # Train and test the model by Multinomial Naive Bayes
    nb_new = MultinomialNB()
    nb_new.fit(train_x, train_y)
    print("The score is :")
    print(nb_new.score(test_x,test_y))
    print()
type1(train_data,train_data_review,test_data,test_data_review)

The score is :
0.862914862914863



## 3. Task 3:

### (a) Explanation of your implementation and experimental setup for task 3.

The polynomial distribution deals with discrete variables, while the other three features are continuous variables except for review. So we try to compute the text word frequency with Bayesian first, and then merge the obtained probability value with other features to continue the computation; and the data of the second computation is not discrete, so we choose to use Gaussian Bayesian for the computation. We also use the char_wb analyzer to reduce the impact of word errors when processing the text word frequencies. We choose to divide the training set into training set and verification set to preliminarily test which parameter can achieve better results, so we finally choose (5,6) as the parameter.

### (b) Summary and discussion of experimental results for task 3.

After the calculation, we obtained a correct rate of about 0.98, which has huge accuracy improvement.

### (c) Attached source codes for task 3.

In [7]:
# reading the data
PATH_ROOT = os.getcwd()
PATH_TRAIN = os.path.join(PATH_ROOT, 'train.csv')
train_data = pd.read_csv(PATH_TRAIN,  header=0)
PATH_TEST = os.path.join(PATH_ROOT, 'test.csv')
test_data = pd.read_csv(PATH_TEST, header=0)
train_data = train_data[['latitude','longitude','mean_checkin_time','review','category']]
test_data = test_data[['latitude','longitude','mean_checkin_time','review','category']]

#从训练集中划分验证集
train_data:pd.DataFrame = train_data.sample(frac=1.0)                     #将数据打乱
rows, cols = train_data.shape
split_index_1 = int(rows * 0.3)

#数据分割
test_data:pd.DataFrame = train_data.iloc[0: split_index_1, :].reset_index()
train_data:pd.DataFrame = train_data.iloc[split_index_1: rows, :].reset_index()

train_data.head()
classes = train_data['category'].unique()
classes_types = CategoricalDtype(categories=classes)
train_data['category'] = train_data['category'].astype(classes_types).cat.codes.astype('long')
test_data['category'] = test_data['category'].astype(classes_types).cat.codes.astype('long')
train_data_review = train_data['review']
test_data_review = test_data['review']

def type3(train_data,train_data_review,test_data,test_data_review, ngram_range):
    def process_review_CountVectorizer(data_review, ngram_range):
        '''
        Training with char_wb analyzer and 6 gram.
        '''
        vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=ngram_range)
        X = vectorizer.fit_transform(data_review)
        T = vectorizer.get_feature_names()
        X = pd.DataFrame(X.toarray(),columns=T)
        X.index.name ='merge_index_unique'
        return X,T

    train_y = train_data['category'].to_numpy()
    test_y = test_data['category'].to_numpy()
    
    new1,T1 = process_review_CountVectorizer(train_data_review, ngram_range)
    new2,T2 = process_review_CountVectorizer(test_data_review, ngram_range)
    T = set(T1)&set(T2)
    T = list(T)
    new1 = new1[T]
    # Train the model with word frequency by Multinomial Naive Bayes, and get the Probability for each reasult as features
    nb1 = MultinomialNB()
    nb1.fit(new1, train_y)
    a = nb1.predict_proba(new1)
    dic ={
    '0': a[:,0],
    '1': a[:,1],
    '2': a[:,2]
    }
    new = pd.DataFrame(dic)
    new.index.name ='merge_index_unique'
    train_data.index.name ='merge_index_unique'   
    train_data_new = pd.merge(train_data.reset_index(),new.reset_index(),on='merge_index_unique')

    new2 = new2[T]
    nb2 = MultinomialNB()
    nb2.fit(new2, test_y)
    a = nb2.predict_proba(new2)
    dic ={
    '0': a[:,0],
    '1': a[:,1],
    '2': a[:,2]
    }
    new = pd.DataFrame(dic)
    new.index.name ='merge_index_unique'
    test_data.index.name ='merge_index_unique'
    test_data_new = pd.merge(test_data.reset_index(),new.reset_index(),on='merge_index_unique')

    train_x = train_data_new.drop(['category','review'], axis=1)
    test_x = test_data_new.drop(['category','review'], axis=1)

    train_x = (train_x-train_x.min())/(train_x.max()-train_x.min())
    test_x = (test_x-test_x.min())/(test_x.max()-test_x.min())

    train_x = train_x.to_numpy()
    test_x = test_x.to_numpy()

    nb_new = GaussianNB()
    nb_new.fit(train_x, train_y)
    print("The score is :",end="  ")
    print(nb_new.score(test_x,test_y))


for i in range(1,8):
    for j in range(i,i+3):
        print(i,j,end="  ")
        type3(train_data,train_data_review,test_data,test_data_review,(i,j))


1 1  The score is :  0.33681765389082463
1 2  The score is :  0.627177700348432
1 3  The score is :  0.5609756097560976
2 2  The score is :  0.7770034843205574
2 3  The score is :  0.7537746806039489
2 4  The score is :  0.7630662020905923
3 3  The score is :  0.8408826945412311
3 4  The score is :  0.8292682926829268
3 5  The score is :  0.8315911730545877
4 4  The score is :  0.8908246225319396
4 5  The score is :  0.8873403019744484
4 6  The score is :  0.8850174216027874
5 5  The score is :  0.9036004645760743
5 6  The score is :  0.8966318234610917
5 7  The score is :  0.8954703832752613
6 6  The score is :  0.9070847851335656
6 7  The score is :  0.908246225319396
6 8  The score is :  0.8931475029036005
7 7  The score is :  0.89198606271777
7 8  The score is :  0.8850174216027874
7 9  The score is :  0.8722415795586528


In [8]:
# reading the data
PATH_ROOT = os.getcwd()
PATH_TRAIN = os.path.join(PATH_ROOT, 'train.csv')
train_data = pd.read_csv(PATH_TRAIN,  header=0)
PATH_TEST = os.path.join(PATH_ROOT, 'test.csv')
test_data = pd.read_csv(PATH_TEST, header=0)
train_data = train_data[['latitude','longitude','mean_checkin_time','review','category']]
test_data = test_data[['latitude','longitude','mean_checkin_time','review','category']]
train_data.head()
classes = train_data['category'].unique()
classes_types = CategoricalDtype(categories=classes)
train_data['category'] = train_data['category'].astype(classes_types).cat.codes.astype('long')
test_data['category'] = test_data['category'].astype(classes_types).cat.codes.astype('long')
train_data_review = train_data['review']
test_data_review = test_data['review']

def type4(train_data,train_data_review,test_data,test_data_review, ngram_range):
    def process_review_CountVectorizer(data_review, ngram_range):
        '''
        Training with char_wb analyzer and gram.
        '''
        vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=ngram_range)
        X = vectorizer.fit_transform(data_review)
        T = vectorizer.get_feature_names()
        X = pd.DataFrame(X.toarray(),columns=T)
        X.index.name ='merge_index_unique'
        return X,T

    train_y = train_data['category'].to_numpy()
    test_y = test_data['category'].to_numpy()
    # 分别对训练集和测试集进行词频处理
    new1,T1 = process_review_CountVectorizer(train_data_review, ngram_range)
    new2,T2 = process_review_CountVectorizer(test_data_review, ngram_range)
    # 得到词频交集
    T = set(T1)&set(T2)
    T = list(T)
    new1 = new1[T]
    
    # Train the model with word frequency by Multinomial Naive Bayes, and get the Probability for each reasult as features
    nb1 = MultinomialNB()
    nb1.fit(new1, train_y)
    a = nb1.predict_proba(new1)
    dic ={
    '0': a[:,0],
    '1': a[:,1],
    '2': a[:,2]
    }
    new = pd.DataFrame(dic)
    new.index.name ='merge_index_unique'
    train_data.index.name ='merge_index_unique'
    train_data_new = pd.merge(train_data.reset_index(),new.reset_index(),on='merge_index_unique')

    new2 = new2[T]

    nb2 = MultinomialNB()
    nb2.fit(new2, test_y)
    a = nb2.predict_proba(new2)
    dic ={
    '0': a[:,0],
    '1': a[:,1],
    '2': a[:,2]
    }
    new = pd.DataFrame(dic)
    new.index.name ='merge_index_unique'
    test_data.index.name ='merge_index_unique'
    test_data_new = pd.merge(test_data.reset_index(),new.reset_index(),on='merge_index_unique')

    train_x = train_data_new.drop(['category','review'], axis=1)
    test_x = test_data_new.drop(['category','review'], axis=1)

    train_x = (train_x-train_x.min())/(train_x.max()-train_x.min())
    test_x = (test_x-test_x.min())/(test_x.max()-test_x.min())

    train_x = train_x.to_numpy()
    test_x = test_x.to_numpy()
    nb_new = GaussianNB()
    nb_new.fit(train_x, train_y)
    print("The score is :")
    print(nb_new.score(test_x,test_y))
    print()
  
type4(train_data,train_data_review,test_data,test_data_review,(5,6))

The score is :
0.9812409812409812

